**Problem definition**

In this notebook we will determine the relative popularity of the syrup flavors, both overall and locally. This should give us insight into which flavors can be cut or replaced. We will also discuss a way to recommend new flavors that haven’t been created yet.

In [1]:
import os
import numpy as np
import pandas as pd
import surprise # scikit-surprise is a scipy toolkit for building recommender systems
data_folder_name = "data"
pd.options.mode.chained_assignment = None #to silence warning msgs

In [2]:
main_df = pd.read_csv(os.path.join('data', 'joined_data.csv'), index_col = 'entry_num')
main_df['flav1_name'] = None
main_df['flav2_name'] = None
main_df['flav3_name'] = None

main_df = main_df.astype({'flavor_name': 'category','timestamp':'datetime64[ns]', 'station_id': 'category',
                          'flav1_lvl_after': 'int16', 'flav2_lvl_after': 'int16', 'flav3_lvl_after': 'int16'})

Each entry in our dataset contains data about all 3 flavors levels and the name of flavor currently being sold. It does not however explicitly specify the names of other two flavors present in the dispenser at any given time. That information is however present implicitly, so we will extract that information and populate flavor name data 

In [3]:
# not the most efficient code, but it does the job within reasonable amount of time
print('populating flavor labels for station IDs: ')
for st_id in main_df['station_id'].unique(): 
    print(st_id, end = ' ')
    temp = main_df[main_df.station_id == st_id]
    start_ind = temp.iloc[0].name 
    cur_flavors = [None, None, None]
    backfill_from = [start_ind, start_ind, start_ind] 
    
    for i in range(start_ind, start_ind + len(temp)):
        if i == start_ind:
            prev_lvls = [500]*3
        else:
            prev_lvls = [temp.loc[i-1]['flav1_lvl_after'], temp.loc[i-1]['flav2_lvl_after'], temp.loc[i-1]['flav3_lvl_after']]

        cur_lvls = [temp.loc[i]['flav1_lvl_after'], temp.loc[i]['flav2_lvl_after'], temp.loc[i]['flav3_lvl_after']]
        lvl_delta = list(map(lambda x, y: x - y, prev_lvls, cur_lvls))
        flav_used_ind = lvl_delta.index(max(lvl_delta))

        if min(lvl_delta) < 0: #refill event
            cur_flavors[lvl_delta.index(min(lvl_delta))] = None
            backfill_from[lvl_delta.index(min(lvl_delta))] = i

        prev_flavor = cur_flavors[flav_used_ind] 

        if temp.loc[i]['flavor_name'] != 'water':
            cur_flavors[flav_used_ind] = temp.loc[i]['flavor_name']
            
        if prev_flavor != cur_flavors[flav_used_ind] and backfill_from[flav_used_ind] is not None:
            temp.loc[backfill_from[flav_used_ind] : i]['flav' + str(flav_used_ind + 1) + '_name'] = cur_flavors[flav_used_ind]
            backfill_from[flav_used_ind] = None
        temp.at[i, 'flav1_name'], temp.at[i, 'flav2_name'], temp.at[i, 'flav3_name'] = cur_flavors

    main_df.update(temp)

main_df.to_csv(os.path.join('data', 'joined_data_with_flav_names.csv')) 

populating flavor labels for station IDs: 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 

The approach shown above will fill *almost* all flavor names but it will not be able to populate names for one specific edge. If one of the flavors was restocked and never ordered again after being restocked - it's name will not be populated. This, however, affects only a small number of data points (typically - several last entries near the end of the time interval present in a dataset, for a given station_id), which can be considered acceptable.

In [4]:
main_df = pd.read_csv(os.path.join('data', 'joined_data_with_flav_names.csv'), index_col = 'entry_num')

In [5]:
main_df.head()

,amount_vended_L,flavor_id,flavor_levels_after,flavor_name,tank_level_after,timestamp,station_id,flav1_lvl_after,flav2_lvl_after,flav3_lvl_after,country,tank_size,flav1_name,flav2_name,flav3_name
entry_num,,,,,,,,,,,,,,,
0,1.0,22.0,"['499', '500', '500']",mint,499.0,2019-01-01 02:30:35.185654,0,499.0,500.0,500.0,UTO,500.0,mint,pear,strawberry
1,0.5,22.0,"['498', '500', '500']",mint,498.5,2019-01-01 03:35:02.746804,0,498.0,500.0,500.0,UTO,500.0,mint,pear,strawberry
2,1.0,20.0,"['498', '500', '499']",strawberry,497.5,2019-01-01 04:13:23.046128,0,498.0,500.0,499.0,UTO,500.0,mint,pear,strawberry
3,1.0,22.0,"['497', '500', '499']",mint,496.5,2019-01-01 04:38:28.985597,0,497.0,500.0,499.0,UTO,500.0,mint,pear,strawberry
4,1.0,13.0,"['497', '499', '499']",pear,495.5,2019-01-01 05:03:36.390141,0,497.0,499.0,499.0,UTO,500.0,mint,pear,strawberry


Now, need to come up with a score function, which will be used to determine the best (most preferred) and the worst flavors. There are several ways to do it, our approach will be based on relative ranking of flavors for each entry: 
* if a flavor is available and selected by the user: score = 2
* if a flavor is available and not selected by the user: score = -1
* if a flavor is not available: score value is not assigned
* if the dispensed liquid is plain water: score values are not assigned for all 3 flavors in this entry

In [6]:
# we will not score water - it doesn't compete for dispenser slots with flavored syrups
def score_flavor(flavor_scored_name, flavor_sold_name, flavor_scored_lvl):
    if flavor_sold_name == 'water':
        return np.nan 
    if flavor_scored_name == flavor_sold_name: # user preferred this flavor compared to other available ones
        return 2
    if flavor_scored_lvl > 0: # user didn't pick the flavor even if it was available
        return -1 
    else: # flavor wasn't selected by user, but its level was zero so it was technically unavailable
        return np.nan

main_df['flav1_score'] = main_df.apply(lambda x: score_flavor(x.flav1_name, x.flavor_name, x.flav1_lvl_after), axis=1)
main_df['flav2_score'] = main_df.apply(lambda x: score_flavor(x.flav2_name, x.flavor_name, x.flav2_lvl_after), axis=1)
main_df['flav3_score'] = main_df.apply(lambda x: score_flavor(x.flav3_name, x.flavor_name, x.flav3_lvl_after), axis=1)

In [7]:
main_df.head(15)

,amount_vended_L,flavor_id,flavor_levels_after,flavor_name,tank_level_after,timestamp,station_id,flav1_lvl_after,flav2_lvl_after,flav3_lvl_after,country,tank_size,flav1_name,flav2_name,flav3_name,flav1_score,flav2_score,flav3_score
entry_num,,,,,,,,,,,,,,,,,,
0,1.0,22.0,"['499', '500', '500']",mint,499.0,2019-01-01 02:30:35.185654,0,499.0,500.0,500.0,UTO,500.0,mint,pear,strawberry,2.0,-1.0,-1.0
1,0.5,22.0,"['498', '500', '500']",mint,498.5,2019-01-01 03:35:02.746804,0,498.0,500.0,500.0,UTO,500.0,mint,pear,strawberry,2.0,-1.0,-1.0
2,1.0,20.0,"['498', '500', '499']",strawberry,497.5,2019-01-01 04:13:23.046128,0,498.0,500.0,499.0,UTO,500.0,mint,pear,strawberry,-1.0,-1.0,2.0
3,1.0,22.0,"['497', '500', '499']",mint,496.5,2019-01-01 04:38:28.985597,0,497.0,500.0,499.0,UTO,500.0,mint,pear,strawberry,2.0,-1.0,-1.0
4,1.0,13.0,"['497', '499', '499']",pear,495.5,2019-01-01 05:03:36.390141,0,497.0,499.0,499.0,UTO,500.0,mint,pear,strawberry,-1.0,2.0,-1.0
5,1.0,13.0,"['497', '498', '499']",pear,494.5,2019-01-01 05:21:31.048165,0,497.0,498.0,499.0,UTO,500.0,mint,pear,strawberry,-1.0,2.0,-1.0
6,1.0,13.0,"['497', '497', '499']",pear,493.5,2019-01-01 05:39:25.540227,0,497.0,497.0,499.0,UTO,500.0,mint,pear,strawberry,-1.0,2.0,-1.0
7,1.0,22.0,"['496', '497', '499']",mint,492.5,2019-01-01 05:57:24.621983,0,496.0,497.0,499.0,UTO,500.0,mint,pear,strawberry,2.0,-1.0,-1.0
8,0.5,20.0,"['496', '497', '498']",strawberry,492.0,2019-01-01 06:15:19.236103,0,496.0,497.0,498.0,UTO,500.0,mint,pear,strawberry,-1.0,-1.0,2.0


Now we'll reshape this dataset so that it can be used as an input into our recommendation system. Each row in the new dataset will consist of a single flavor 'review' containing:
* station_id ("reviewer")
* flavor_id ("reviewed product")
* review score

This will allow us to average scores for each station-flavor combination and find the most and least popular flavors

In [8]:
temp1 = main_df[['station_id','flav1_name','flav1_score']]
temp1.rename(columns={"flav1_name": "flavor", "flav1_score": "score"}, inplace = True)

temp2 = main_df[['station_id','flav2_name','flav2_score']]
temp2.rename(columns={"flav2_name": "flavor", "flav2_score": "score"}, inplace = True)

temp3 = main_df[['station_id','flav3_name','flav3_score']]
temp3.rename(columns={"flav3_name": "flavor", "flav3_score": "score"}, inplace = True)

review_df = pd.concat([temp1, temp2, temp3], ignore_index = True)
review_df.dropna(inplace = True)

review_df.to_csv(os.path.join('data','reviews.csv'))

review_df.head()

,station_id,flavor,score
0,0,mint,2.0
1,0,mint,2.0
2,0,mint,-1.0
3,0,mint,2.0
4,0,mint,-1.0


In [9]:
aver_reviews = review_df.pivot_table(index = ['station_id'], columns = 'flavor', aggfunc = np.mean, fill_value = np.nan)

First, let's see which flavors were the most in-demand overall:

In [10]:
aver_reviews.mean().sort_values(ascending = False)

       flavor       
score  raspberry        1.091386
       lemon_lime       1.068529
       grapefruit       1.036371
       lemon            0.952281
       cranberry        0.895815
       lime             0.894584
       mango            0.876989
       green_tea        0.853808
       lemonade         0.834778
       blackberry       0.799866
       coconut          0.781182
       pear             0.748107
       orange           0.738859
       oolong           0.703679
       acai             0.657931
       cherry           0.601435
       pomegranate      0.595812
       earl_grey_tea    0.551342
       pine_needle      0.506344
       strawberry       0.445821
       blueberry        0.444513
       cucumber         0.443733
       peach            0.443428
       eucalyptus       0.373027
       mint             0.318567
       key_lime         0.235439
       tuna_fish       -0.412390
       mackerel        -0.706578
dtype: float64

Looks like mackerel and tuna are near the bottom of the popularity list and could be cut or replaced.
Now, let's look how these preferences vary from station to station:

In [11]:
aver_reviews = aver_reviews.T

#some styling code from https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
def highlight_max(data, color='lawngreen'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

def highlight_min(data, color='red'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_min = data == data.min()
        return [attr if v else '' for v in is_min]
    else:  # from .apply(axis=None)
        is_min = data == data.min().min()
        return pd.DataFrame(np.where(is_min, attr, ''),
                            index=data.index, columns=data.columns)
    


In [12]:
aver_reviews.style.apply(highlight_min)

If we wanted to cut or replace 1 flavor per station, the ones highlighted in red would be a good candidates.
Instead of just using the global or local "leaderboards" as the ones shown below, we can build a simple recommender system using the previously-constructed reviews dataframe as a datasource:

In [13]:
reader = surprise.Reader(rating_scale=(-1, 2))
data = surprise.Dataset.load_from_df(review_df[['station_id', 'flavor', 'score']], reader)

In [14]:
# SVD + 5-fold cross-validation
algorithm = surprise.SVD()
surprise.model_selection.cross_validate(algorithm, data, measures = ['RMSE', 'MAE'], cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3734  1.3215  1.3218  1.3217  1.3268  1.3330  0.0203  
MAE (testset)     1.2276  1.1317  1.1330  1.1309  1.1397  1.1525  0.0376  
Fit time          59.03   59.79   59.28   59.22   59.09   59.28   0.27    
Test time         3.08    3.25    3.09    2.87    2.87    3.03    0.14    


{'test_rmse': array([1.37343099, 1.32147615, 1.32176523, 1.32165695, 1.32680432]),
 'test_mae': array([1.22756981, 1.13165836, 1.13296759, 1.13087321, 1.13966891]),
 'fit_time': (59.033581018447876,
  59.789064168930054,
  59.282092571258545,
  59.218194007873535,
  59.088555335998535),
 'test_time': (3.075010061264038,
  3.2477855682373047,
  3.08791446685791,
  2.870868682861328,
  2.8662800788879395)}

Now we can see how recommender output compares to averages for each station-combo, as well as see an estimate for those combinations that haven't been tried yet in the field:

In [21]:
from random import choice, seed

seed(1337)

stations = list(aver_reviews)
flavors = list(aver_reviews.index.get_level_values(1))

for _ in range(10):
    station_id = choice(stations)  
    flavor = choice(flavors)
    # Inputs: station_id, flavor, (optional) r_ui - "real" score for comparison (we'll use aaver_reviews as a proxy)
    # output: predicted score ("est")
    pred = algorithm.predict(str(station_id), flavor, aver_reviews.loc[('score', flavor), station_id], verbose = True)

user: 39         item: mango      r_ui = 0.65   est = 0.83   {'was_impossible': False}
user: 45         item: key_lime   r_ui = 0.33   est = 0.37   {'was_impossible': False}
user: 36         item: mint       r_ui = nan   est = 0.05   {'was_impossible': False}
user: 46         item: cranberry  r_ui = 0.58   est = 0.72   {'was_impossible': False}
user: 49         item: strawberry r_ui = 1.40   est = 0.61   {'was_impossible': False}
user: 21         item: lemon      r_ui = 1.99   est = 0.57   {'was_impossible': False}
user: 40         item: raspberry  r_ui = 1.05   est = 1.00   {'was_impossible': False}
user: 23         item: grapefruit r_ui = nan   est = 0.71   {'was_impossible': False}
user: 25         item: pear       r_ui = 0.60   est = 0.51   {'was_impossible': False}
user: 13         item: pomegranate r_ui = nan   est = 0.50   {'was_impossible': False}


**Discussion**

Suppose we were tasked to come up with a way to recommend new flavors that haven’t been created yet. 

How would we approach this task? 

One way would to be to map each flavor to a list of characteristics (for example: sour, sweet, bitter, spicy) and use these characteristics/features as model inputs instead of using flavors like "lemon" or "cherry" directly. If we need to introduce a new flavor, we could predict its popularity as long it can be defined in terms of these features.